In [1]:
from cnn_method import *
from cnn_preproc_function import *
from multi_cnn_preproc_function import *
from inference_function import *
from performance_function import *

import optparse
import numpy as np
import pandas as pd

import sklearn
from sklearn import preprocessing

import keras
import tensorflow as tf

from keras_model_configuration import *
from keras_metric import *

import datetime
import time
import os
import shutil

from sklearn.externals import joblib

#OPTS PARSER
# optparser = optparse.OptionParser()
# optparser.add_option("-a", "--assetindex", default=0, help="assetindex")
# optparser.add_option("-d", "--gpudevice", default="1", help="gpudevice")
# opts = optparser.parse_args()[0]

# assetindex = int(opts.assetindex)
# gpudevice = opts.gpudevice
assetindex = 0
gpudevice = "1"

#GPU CONFIG
os.environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"] = "4"
os.environ["CUDA_VISIBLE_DEVICES"]=gpudevice
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

####################
#SETUP INIT
####################
main_folder = "/home/workstation/Desktop/CNN/output/RNN_Multi_1_long_60_5_day"
collect_signal_folder = "/home/workstation/Desktop/CNN/output/signal_RNN_Multi_1_long_60_5_day"
daily = True

batch_size = 64

####################
#END OF SETUP INIT
####################
init_dir(collect_signal_folder)

try:
    setups_folder = [j for j in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder,j))]
    print(setups_folder)
    setup_folder_name = setups_folder[assetindex]
    print("Running:", setup_folder_name)

except Exception as err:
    print("Index:", assetindex, "--SETUP ERROR!")
    raise Exception(err)

setup_path = os.path.join(main_folder, setup_folder_name)
DATA_PARAMS = joblib.load(os.path.join(setup_path, "DATA_PARAMS.pkl"))
MODEL_PARAMS = joblib.load(os.path.join(setup_path, "MODEL_PARAMS.pkl"))
scaler = joblib.load(os.path.join(setup_path, "scaler.pkl"))
locals().update(DATA_PARAMS)
locals().update(MODEL_PARAMS)

if daily:
    df = load_data_daily_close_missing(raw_data_file)
    #Create target
    df, X, Y, x_columns, y_columns = multi_create_target(df, FUTURE_PERIOD_PREDICT, TARGET_FUNCTION, TARGET_THRESHOLD, FLIP, selected_cols = SELECTED_COLS)
    X = scaler.transform(X)
    
    
    all_data_gen = TimeseriesGenerator(X, Y,
                       length=SEQ_LEN, sampling_rate=1,
                       batch_size=BATCH_SIZE,
                       shuffle=False)
    timestamp = df.index.values
    shape_x = all_data_gen[0][0].shape
    shape_y = all_data_gen[0][1].shape
    print(shape_x, shape_y, X.shape, Y.shape, len(timestamp))
    print("Number of batches per epoch:", len(all_data_gen))
    print("TSGenerator: Done!")



Using TensorFlow backend.


['Multi_1540459285']
Running: Multi_1540459285
Load Data: Done!
Clean Data: Done!
(64, 60, 36) (64, 7) (4410, 36) (4410, 7) 4410
Number of batches per epoch: 68
TSGenerator: Done!


In [2]:
#Pick best model
models_folder = os.path.join(setup_path, "models")
mdf = parse_model_folder(models_folder)
fmdf, best_model_file = best_model_score_multi(mdf)
best_model_path = os.path.join(models_folder,best_model_file)
model = keras.models.load_model(best_model_path, custom_objects=None, compile=False)
print("Best Model: Done!")

Best Model: Done!


In [3]:
#Prediction
t0 = time.time()
y = model.predict_generator(all_data_gen)
t1 = time.time()
print("Prediction: Done!", t1-t0, "seconds!")

Prediction: Done! 1.1645996570587158 seconds!


In [4]:
#Remove existing signal file
try:
    os.remove(glob.glob(os.path.join(main_folder, setup_folder_name, "*.csv"))[0])
except:
    print("Error deleting signal.csv:", setup_folder_name)

Error deleting signal.csv: Multi_1540459285


In [5]:
#Signal Output
lower = len(timestamp)-y.shape[0]
Date = timestamp[lower:]
signal_df = pd.DataFrame(y, index = Date, columns = y_columns)
signal_file = "signal_" + setup_folder_name + "_" + str(FLIP) + ".csv"
signal_df.to_csv(os.path.join(setup_path, signal_file))
print("Signal Output: Done!")

Signal Output: Done!


In [6]:
shutil.copy2(best_model_path, setup_path)
print("Copy best model: Done!")

Copy best model: Done!


In [7]:
signal_df.to_csv(os.path.join(collect_signal_folder, signal_file))
print("Collect signal.csv: Done!")


Collect signal.csv: Done!


In [8]:
#Scoring
actual_df = pd.DataFrame(Y[lower:], index = Date, columns = y_columns)
L = np.sum(signal_df.index <= DATA_PARAMS["end_split"][0])
for i in range(y.shape[1]):
    yscore = signal_df.values[L:,i]
    ytrue = actual_df.values[L:,i]
    perf(ytrue,yscore, t = 0.5)
    print("===============", i)
    print("===============", i)

n: 1984
true: prop: 0.5720766129032258 n1: 1135
pred: prop: 0.5846774193548387 n1: 1160
cm:
 [[364 485]
 [460 675]]
prec: 0.5818965517241379
auc: 0.5054383752847351
=============== 0
=============== 0
n: 1984
true: prop: 0.4753024193548387 n1: 943
pred: prop: 0.8684475806451613 n1: 1723
cm:
 [[128 913]
 [133 810]]
prec: 0.4701102727800348
auc: 0.5252907566038447
=============== 1
=============== 1
n: 1984
true: prop: 0.577116935483871 n1: 1145
pred: prop: 0.5529233870967742 n1: 1097
cm:
 [[387 452]
 [500 645]]
prec: 0.5879671832269827
auc: 0.5017243443275682
=============== 2
=============== 2
n: 1984
true: prop: 0.5514112903225806 n1: 1094
pred: prop: 0.48588709677419356 n1: 964
cm:
 [[468 422]
 [552 542]]
prec: 0.5622406639004149
auc: 0.5143910605344781
=============== 3
=============== 3
n: 1984
true: prop: 0.4470766129032258 n1: 887
pred: prop: 0.17338709677419356 n1: 344
cm:
 [[919 178]
 [721 166]]
prec: 0.48255813953488375
auc: 0.534562335117092
=============== 4
=============== 